In [1]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Running time

/Volumes/neuropixel:
    - Alessandro rem-stim2 (56GB): 1h45 (including binfile copying with 20 jobs <20min)

In [2]:
from pathlib import Path
from datetime import datetime
from joblib import Parallel, delayed
import spikeinterface.extractors as se
import spikeinterface.sorters as ss
import spikeinterface.widgets as sw
import spikeinterface.toolkit as st
import itertools

In [3]:
from ecephys_analyses.data import channel_groups, paths, parameters

In [4]:
ss.installed_sorters()

['ironclust', 'kilosort', 'kilosort2_5', 'kilosort3']

In [5]:
## USER

sorting_conditions = [
    'ks2_5_catgt_df'
#     'ks3_catgt_df'
]

# Subject, condition, catgt data
data_conditions = [
    ('Allan', 'g2-1', True),
    ('Allan', 'g2-0', True),
#     ('Alessandro', 'rem-stim', True),
#     ('Alessandro', 'rem-stim2', True),
#     ('Alessandro', 'sleep-stim', True),
#     ('Alessandro', 'West', True),
    
]

n_jobs = 1  # Don't parallelize (possibly messed up tmp directories?)

## end USER

In [6]:
def run_sorting(subject, condition, sorting_condition,
                catgt_data=True, bad_channels=None, rerun_existing=True):
    
    # Sorter parameters
    sorter, params = parameters.get_analysis_params(
        'sorting',
        sorting_condition
    )
    
    # Recording
    rec = prepare_data(
        subject, 
        condition,
        catgt_data=catgt_data,
        bad_channels=bad_channels
    )
    
    # Path 
    output_dir = paths.get_sglx_style_datapaths(
        subject,
        condition,
        'dummy',
        catgt_data=catgt_data
    )[0].parent/sorting_condition
    output_dir.mkdir(exist_ok=True, parents=True)
    print(f"Saving sorting output at {output_dir}")
    
    
    # rerun existing
    if (output_dir/'spike_times.npy').exists() and not rerun_existing:
        print(f'Passing: output directory is done: {output_dir}\n\n')
        return

    print('running')
    start = datetime.now()
    ss.run_sorter(
        sorter,
        rec,
        output_folder=output_dir,
        verbose=True,
        **params
    )

    end = datetime.now()
    print(f"{end.strftime('%H:%M:%S')}: Finished {subject}, {condition}, {sorting_condition}.")
    print(f"Run time = {str(end - start)}\n")


    return 1

def prepare_data(subject, condition, catgt_data=True, bad_channels=None):

    binpaths = paths.get_sglx_style_datapaths(
        subject,
        condition,
        'ap.bin',
        catgt_data=catgt_data
    )
    
    rec_extractors = [
        se.SpikeGLXRecordingExtractor(binpath)
        for binpath in binpaths
    ]
    if len(rec_extractors) == 1:
        recording = rec_extractors[0]
    else:    
        recording = se.MultiRecordingTimeExtractor(extractors)
    
    assign_locations(recording, binpaths[0])

    if bad_channels is not None:
        recording = st.preprocessing.remove_bad_channels(
            recording, bad_channel_ids=bad_channels
        )
        
    return recording

def assign_locations(recording, binpath, plot=False):
    print(binpath)
    from ecephys.sglx_utils import get_xy_coords
    idx, x, y = get_xy_coords(binpath)
    recording.set_channel_locations(
        [(x[i], y[i]) for i in range(len(idx))],
        channel_ids=idx,
    )
    if plot:
        from ecephys.plot import plot_channel_coords
        plot_channel_coords(range(len(x)), x, y)

    


In [7]:
if n_jobs == 1:
    for (
        (subject, condition, catgt_data),
        sorting_condition
    ) in itertools.product(data_conditions, sorting_conditions):
        print(subject, condition, catgt_data, sorting_condition)
        run_sorting(
            subject,
            condition,
            sorting_condition,
            catgt_data=catgt_data,
            bad_channels=None,
            rerun_existing=False
        )
else:
    parallel = Parallel(
        n_jobs=n_jobs,
        backend='multiprocessing',
    )(
        delayed(run_sorting)(
            subject,
            condition,
            sorting_condition,
            catgt_data=catgt_data,
            bad_channels=None,
            rerun_existing=False
        ) for ((subject, condition, catgt_data), sorting_condition)
        in itertools.product(data_conditions, sorting_conditions)
    )

Allan g2-1 True ks2_5_catgt_df
/Volumes/neuropixel/Data/tom/catgt/CNPIX8-Allan/3-5-2021/catgt_3-5-2021_g2/3-5-2021_g2_imec1/3-5-2021_g2_tcat.imec1.ap.bin
0
Saving sorting output at /Volumes/neuropixel/Data/tom/catgt/CNPIX8-Allan/3-5-2021/catgt_3-5-2021_g2/3-5-2021_g2_imec1/ks2_5_catgt_df
running
RUNNING SHELL SCRIPT: sh /Volumes/neuropixel/Data/tom/catgt/CNPIX8-Allan/3-5-2021/catgt_3-5-2021_g2/3-5-2021_g2_imec1/ks2_5_catgt_df/run_kilosort2_5.sh


                            < M A T L A B (R) >

                  Copyright 1984-2019 The MathWorks, Inc.

                  R2019b (9.7.0.1190202) 64-bit (glnxa64)

                              August 21, 2019





                            < M A T L A B (R) >

                  Copyright 1984-2019 The MathWorks, Inc.

                  R2019b (9.7.0.1190202) 64-bit (glnxa64)

                              August 21, 2019



 

 

To get started, type doc.

To get started, type doc.

For product information, visit www.mathworks.com.

For 

KeyboardInterrupt: 